# Azure AI Agent Hizmeti ve Semantic Kernel Kullanarak RAG

Bu kod parçası, `Azure AI Agent Hizmeti` ve `Semantic Kernel` kullanarak bilgi-getirme destekli üretim (RAG) için bir Azure AI ajanı oluşturmayı ve yönetmeyi göstermektedir. Ajan, kullanıcı sorgularını alınan bağlama göre işler ve buna uygun doğru yanıtlar sağlar.


SQLite Sürüm Düzeltmesi  
Eğer şu hatayla karşılaşırsanız:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Not defterinizin başında bu kod bloğunun yorumunu kaldırın:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Paketleri İçe Aktarma
Aşağıdaki kod, gerekli paketleri içe aktarır:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Semantic Kernel ve Azure AI Agent Hizmeti ile Bilgi Getirme Destekli Üretim

Bu örnek, bir dil modelini yüklenen bir belgeden alınan alanına özgü bağlamla birleştirerek **Bilgi Getirme Destekli Üretim (RAG)** işlemini gerçekleştirmek için **Azure AI Agent Hizmeti**nin nasıl kullanılacağını göstermektedir.

### Nasıl Çalışır?

1. **Belge Yükleme**: Bir markdown dosyası (document.md) içeriğinde bilgi (Contoso'nun seyahat sigortası politikası) ile birlikte agent hizmetine yüklenir.

2. **Vektör Deposu Oluşturma**: Belge, içeriği üzerinde anlamsal arama yapılmasını sağlamak için bir vektör deposuna indekslenir.

3. **Agent Yapılandırması**: `gpt-4o` modeli kullanılarak bir agent aşağıdaki katı talimatlarla oluşturulur:
   - Sadece belgeden alınan içeriklere dayalı sorulara cevap ver.
   - Soru kapsam dışındaysa cevap vermeyi reddet.

4. **Dosya Arama Aracı Entegrasyonu**: `FileSearchTool`, agent ile kaydedilir ve modelin çıkarım sırasında indekslenmiş belgeden ilgili parçaları arayıp almasını sağlar.

5. **Kullanıcı Etkileşimi**: Kullanıcılar sorular sorabilir. Eğer belgede ilgili bilgi bulunursa, agent dayanaklı bir cevap üretir.  
   Eğer bilgi bulunamazsa, agent açıkça belgenin yeterli bilgi içermediğini belirtir.


### Ana Fonksiyon



Azure CLI kullanarak `DefaultAzureCredential` kullanırken doğru kimlik doğrulama bağlamının sağlanması için önce `az login` çalıştırdığınızdan emin olun. Azure AI Agent Hizmeti API anahtarlarını kullanmaz.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Anladım! Lütfen çevirisini yapmamı istediğiniz Markdown dosyasını paylaşın.



---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan herhangi bir yanlış anlama veya yanlış yorumlama durumunda sorumluluk kabul edilmez.
